In [ ]:
# Import TensorFlow and NumPy
import tensorflow as tf
import numpy as np
from PINNs_Chron import PINN1D, PINN1DSolver

import matplotlib.pyplot as plt

# Set data type
DTYPE='float32'
tf.keras.backend.set_floatx(DTYPE)
tf.get_logger().setLevel('ERROR')

In [ ]:
%matplotlib inline

SIZE = 12
BIGGER_SIZE = 16

plt.rc('font', family='Arial', size=SIZE) # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize

In [ ]:
def logistic_f(x, a=0, L=1, k=1, x0=0):
    return a + L / (1 + np.exp(-k * (x - x0)))

tf_logistic = lambda t, a, L, k, x0: a + L * tf.math.sigmoid(k * (t - x0))

In [ ]:
t_end = 50.
h = 20.
Tsurf = 0.
Tbot = 500.
Tgrad0 = (Tbot - Tsurf) / h
kappa = 25.
u0, u1, t1 = .05, .8, 35
uplift = lambda t : logistic_f(t, a=u0, L=u1, k=2, x0=t1)
tf_uplift = lambda t: tf_logistic(t, u0, u1, 2, t1)
tf_uplift_inv = lambda t, u0, u1, t1: tf_logistic(t, u0, u1, 2, t1) 

In [ ]:
# Define the initial condition
def T_init(z):
    return Tbot - Tgrad0 * z

# Define boundary condition
def T_surf(z):
    return tf.constant(Tsurf, shape=z.shape, dtype=DTYPE)

def T_bot(z):
    return tf.constant(Tbot, shape=z.shape, dtype=DTYPE)

In [ ]:
# Set number of data points
N_0 = 50
N_b = 50
N_r = 1000

# Set boundary
tmin = 0.
tmax = t_end
zmin = 0.
zmax = h

# Lower bounds
lb = tf.constant([tmin, zmin], dtype=DTYPE)
# Upper bounds
ub = tf.constant([tmax, zmax], dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Draw uniform sample points for initial boundary data
t_0 = tf.ones((N_0, 1), dtype=DTYPE) * lb[0]
z_0 = tf.random.uniform((N_0, 1), lb[1], ub[1], dtype=DTYPE)
X_0 = tf.concat([t_0, z_0], axis=1)

# Evaluate intitial condition at z_0
T_0 = T_init(z_0)

# Boundary data
t_b = tf.random.uniform((N_b*2, 1), lb[0], ub[0], dtype=DTYPE)
z_b = lb[1] * tf.ones((N_b, 1), dtype=DTYPE)
z_s = ub[1] * tf.ones((N_b, 1), dtype=DTYPE)
z_bs = tf.concat([z_b, z_s], axis=0)
X_b = tf.concat([t_b, z_bs], axis=1)

# Evaluate boundary condition at (t_b, z_b)
T_b = T_bot(z_b)
T_s = T_surf(z_s)
T_bs = tf.concat([T_b, T_s], axis=0)

# Draw uniformly sampled collocation points
t_r = tf.random.uniform((N_r, 1), lb[0], ub[0], dtype=DTYPE)
z_r = tf.random.uniform((N_r, 1), lb[1], ub[1], dtype=DTYPE)
X_r = tf.concat([t_r, z_r], axis=1)

# Collect boundary and inital data in lists
X_data = tf.concat([X_0, X_b], axis=0)
T_data = tf.concat([T_0, T_bs], axis=0)

In [ ]:
class PINNInv1D(PINN1D):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.u0 = tf.constant(1.)
        self.u1 = tf.constant(1.)
        self.t1= tf.constant(1.)

In [ ]:
from scipy.optimize import differential_evolution as diff_evo
class InvSolver(PINN1DSolver):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.age_loss_hist = []
        self.u0_hist = []
        self.u1_hist = []
        self.t1_hist = []
        self.it_hist = []
        #the following needs to be set up for optimization
        self.data_age = tf.constant([])
        self.data_method = []
        self.data_radi = []
        self.data_err = []
        
    def fun_u(self, t):
        #this defines the model parameters to optimize
        return tf_uplift_inv(t, self.model.u0, self.model.u1, self.model.t1)
            
    def age_loss(self):
        age_pred = []
        sample_t, sample_T = self.get_tT()
        for hm, hr in zip(self.data_method, self.data_radi):
            age_pred.append(self.pred_age(sample_t, sample_T, method=hm, grain_radius=hr))
            
        return tf.reduce_mean(tf.square((age_pred - self.data_age)/self.data_err))
            
    def solve_with_DE(self, TF_optimizer, X, T, bounds, adam_n=100,
                      echofreq=1000, savefreq=1000,
                      strategy='best2bin', init='sobol', disp=True,
                      mutation=(0.5, 1), recombination=0.7,
                      maxiter=100, popsize=30, polish=False):
    
        def DE_obj_fn(x):
            u0, u1, t1 = x[0], x[1], x[2]
            self.model.u0 = u0
            self.model.u1 = u1
            self.model.t1 = t1
            # adam_n = tf.where(solver.iter<10000, 1000, 100).numpy()
            self.solve_with_Adam(TF_optimizer, X, T, N=adam_n, echofreq=echofreq, savefreq=savefreq)
            age_loss = self.age_loss()
            self.u0_hist.append(u0)
            self.u1_hist.append(u1)
            self.t1_hist.append(t1)
            self.it_hist.append(self.iter)
            self.current_age_loss = age_loss.numpy()
            self.age_loss_hist.append(self.current_age_loss)
            return age_loss

        def DEcallback(x, convergence=0.):
            print('Best solution: u0 = {:5.4g} u1 = {:5.4g} t1 = {:6.4g}'.format(
                                  x[0], x[1], x[2]))
            return solver.current_loss < 10 and solver.current_age_loss < .1

        return diff_evo(DE_obj_fn, bounds, strategy=strategy, init=init,
                   disp=disp, maxiter=maxiter, popsize=popsize, polish=polish,
                   mutation=mutation, recombination=recombination,
                   callback=DEcallback)

In [ ]:
def read_ages(txt, DYTPE='float32'):
    ar = np.loadtxt(txt, dtype='str')
    methods = ar[:, 0]
    ages = tf.constant(ar[:, 1].astype(DTYPE))
    radi = ar[:, 2].astype(DTYPE)
    return methods, ages, radi

In [ ]:
# Initialize model
model = PINNInv1D(lb, ub, num_hidden_layers=3,
                   num_neurons_per_layer=20)
model.build(input_shape=(None, 2))

lr = 1e-3
# lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([100e3],[1e-3, 1e-4])
optim = tf.keras.optimizers.Adam(learning_rate=lr)

# Initilize PINN solver
solver = InvSolver(model, X_r)
solver.savepath = 'saved_inv_model'
solver.data_method, solver.data_age, solver.data_radi = read_ages('1d_ages.txt')
solver.data_err = solver.data_age * .1

In [ ]:
from time import time # Initialize model
# Start timer
tic = time()

bounds = [(0, 2), (0, 2), (0, 50)]

# adam_n, de_popsize, de_maxit = 10, 30, 100
adam_n, de_popsize, de_maxit = 10, 15, 200
# adam_n, de_popsize, de_maxit = 50, 10, 80

res = solver.solve_with_DE(optim, X_data, T_data, bounds, mutation=.5, recombination=.5,
                           adam_n=adam_n, popsize=de_popsize, maxiter=de_maxit)

# Print computation time
print('\nComputation time: {} seconds'.format(time()-tic));

In [ ]:
path = 'saved_inv_model/'+ solver.runname + '/'
tf.saved_model.save(solver.model, path)

np.save(path + 'u0_hist', solver.u0_hist)
np.save(path + 'u1_hist', solver.u1_hist)
np.save(path + 't1_hist', solver.t1_hist)
np.save(path + 'it_hist', solver.it_hist)
np.save(path + 'loss_hist', solver.loss_hist)
np.save(path + 'heat_hist', solver.heat_hist)
np.save(path + 'age_loss_hist', solver.age_loss_hist)
np.save(path + 'bound_hist', solver.bound_hist)
np.save(path + 'X_r', X_r)
np.save(path + 'adam_m', adam_n)
np.save(path + 'de_popsize', de_popsize)
np.save(path + 'de_maxit', de_maxit)